In [82]:
import pandas as pd
import re
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
# from textblob import TextBlob

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Quang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Quang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [83]:
data = pd.read_csv("Train.csv", sep=',')

In [84]:
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s',
              'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']
STOPWORDS = set(stopwordlist)

In [85]:
tweet = 'This is happiness cute. the about tacos ran crying #thisisus @nbcthisisus https://t.co/ndxqyl4gjk AKJSBG !!! on !! ???? 1029468798346y 368258 .,/, $#%$'
def preprocess_string(str):
    lemmatizer = WordNetLemmatizer()
    porter = PorterStemmer()

    str = str.lower()
    str = re.sub(r'@[^\s]+','',str)
    str = re.sub(r'((www.[^s]+)|(https?://[^\s]+))',' ',str)
    str = re.sub(r'[^a-zA-Z!?#S]',' ',str)
    str = re.sub(r'\b\w{1,2}\b', '', str)
    str = re.sub(r'!', ' ! ', str)
    str = re.sub(r'\?', ' ? ', str)
    str = " ".join([word for word in str.split() if word not in STOPWORDS])
    str = str.split()
    # blob_object = TextBlob(str)
    # str = blob_object.words (this removes all puncuations)

    # str = [lemmatizer.lemmatize(x) for x in str]
    str = [porter.stem(x) for x in str]
    str = ' '.join(str)
    return str

print(preprocess_string(tweet))

data['text'] = data['text'].apply(lambda text: preprocess_string(text))
# data.to_csv('preprocessed-train-data.csv', index=False)

X = data.text
y = data.sentiment
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=76)



happi cute taco ran cri #thisisu akjsbg ! ! ! ! ! ? ? ? ? #


In [86]:
# list_of_sent = list(X_train_raw)

# words = {}
# for el in list_of_sent:
#     el = el.split()
#     for el2 in el:
#         if el2 not in words:
#             words[el2] = 1
#         else:
#             words[el2] += 1

# for (k, v) in words.items():
#     if v <= 1:
#         print (k, v)
# def remove_single_appearance_word(sent, words):
#     sent = sent.split()
#     for el in sent:
#         if el not in words:
#             sent.remove(el)
#         if el in words:
#             if words[el] <= 5 :
#                 sent.remove(el)
#     return ' '.join(sent)

# X_train_raw = X_train_raw.apply(remove_single_appearance_word, args=(words,))


BOW

In [87]:
from sklearn.feature_extraction.text import CountVectorizer

BoW_vectorizer = CountVectorizer(max_df=0.80, min_df=7)

#Build the feature set (vocabulary) and vectorise the Tarin dataset using BoW
X_train_BoW = BoW_vectorizer.fit_transform(X_train_raw)

#Use the feature set (vocabulary) from Train to vectorise the Test dataset 
X_test_BoW = BoW_vectorizer.transform(X_test_raw)

print("Train feature space size (using BoW):",X_train_BoW.shape)
print("Test feature space size (using BoW):",X_test_BoW.shape)

output_dict = BoW_vectorizer.vocabulary_
output_pd = pd.DataFrame(list(output_dict.items()),columns = ['word','count'])

output_pd.T.to_csv('BoW-vocab.csv',index=False)

Train feature space size (using BoW): (20711, 3432)
Test feature space size (using BoW): (1091, 3432)


In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=5, max_features=5000)

#Build the feature set (vocabulary) and vectorise the Tarin dataset using TFIDF
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_raw)

#Use the feature set (vocabulary) from Train to vectorise the Test dataset 
X_test_tfidf = tfidf_vectorizer.transform(X_test_raw)

print("Train feature space size (using TFIDF):",X_train_tfidf.shape)
print("Test feature space size (using TFIDF):",X_test_tfidf.shape)

Train feature space size (using TFIDF): (20711, 4286)
Test feature space size (using TFIDF): (1091, 4286)


In [89]:
test_data = pd.read_csv('Test.csv', sep=',')
test_data['text'] = test_data['text'].apply(lambda text: preprocess_string(text))
X_real_test_raw = test_data.text
X_real_test_tfidf = tfidf_vectorizer.transform(X_real_test_raw)

print(test_data)

                      id                                               text
0     802217876644052000  putin abduct ukrainian citizen occupi territor...
1     802425296955682000  putin want interven would donat clinton founda...
2     805664502515662000                                  elsewher progland
3     639928670103015000  femal cop gucci foamposit tomorrow ? checkout ...
4     673824182287904000  news great saturday town old uni friend david ...
...                  ...                                                ...
6094  662664783083122000  ticket see vamp april tri fan project sign hold ?
6095  635582180316479000  may seen but john cena given make wish gift ki...
6096  805537394262994000                     democraci look like like share
6097  640586501571637000  damn shawn right may conced win alon even numb...
6098  802428434588827000  white liber make sick like ancestor weren dict...

[6099 rows x 2 columns]


In [90]:
from sklearn.svm import SVC,LinearSVC
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

SVMmodel = SVC()
o_vs_r = OneVsOneClassifier(SVMmodel)

# o_vs_r.fit(X_train_BoW, y_train)
# y_pred_svm_BoW = o_vs_r.predict(X_test_BoW)

# cf=classification_report(y_test,y_pred_svm_BoW)
# print(cf)

o_vs_r.fit(X_train_tfidf, y_train)
y_pred_svm_tfidf = o_vs_r.predict(X_test_tfidf)

cf=classification_report(y_test,y_pred_svm_tfidf)
print(cf)

y_pred_real_svm_tfidf = o_vs_r.predict(X_real_test_tfidf)
data = {'id':test_data.id,'text':X_real_test_raw,'sentiment':y_pred_real_svm_tfidf}
predictions = pd.DataFrame(data=data)
(predictions.drop(['text'], axis=1)).to_csv('svm-model-tfidf.csv', index=False)

              precision    recall  f1-score   support

    negative       0.76      0.26      0.39       181
     neutral       0.69      0.91      0.79       659
    positive       0.72      0.46      0.56       251

    accuracy                           0.70      1091
   macro avg       0.72      0.54      0.58      1091
weighted avg       0.71      0.70      0.67      1091



In [91]:
from sklearn.linear_model import LogisticRegression
LRmodel = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1, multi_class='multinomial', solver='lbfgs')

# LRmodel.fit(X_train_BoW, y_train)
# y_pred_svm_BoW = LRmodel.predict(X_test_BoW)

# cf=classification_report(y_test,y_pred_svm_BoW)
# print(cf)

LRmodel.fit(X_train_tfidf, y_train)
y_pred_svm_tfidf = LRmodel.predict(X_test_tfidf)

cf=classification_report(y_test,y_pred_svm_tfidf)
print(cf)


              precision    recall  f1-score   support

    negative       0.66      0.39      0.49       181
     neutral       0.72      0.86      0.78       659
    positive       0.67      0.55      0.60       251

    accuracy                           0.71      1091
   macro avg       0.69      0.60      0.63      1091
weighted avg       0.70      0.71      0.69      1091



In [92]:
from sklearn.naive_bayes import MultinomialNB
BNBmodel = MultinomialNB()

# BNBmodel.fit(X_train_BoW, y_train)
# y_pred_svm_BoW = BNBmodel.predict(X_test_BoW)

# cf=classification_report(y_test,y_pred_svm_BoW)
# print(cf)

BNBmodel.fit(X_train_tfidf, y_train)
y_pred_svm_tfidf = BNBmodel.predict(X_test_tfidf)

cf=classification_report(y_test,y_pred_svm_tfidf)
print(cf)

              precision    recall  f1-score   support

    negative       0.70      0.20      0.32       181
     neutral       0.67      0.91      0.77       659
    positive       0.67      0.37      0.48       251

    accuracy                           0.67      1091
   macro avg       0.68      0.50      0.52      1091
weighted avg       0.67      0.67      0.63      1091



In [93]:
y_pred_real_lr_tfidf = LRmodel.predict(X_real_test_tfidf)
data = {'id':test_data.id,'text':X_real_test_raw,'sentiment':y_pred_real_lr_tfidf}
predictions = pd.DataFrame(data=data)
(predictions.drop(['text'], axis=1)).to_csv('lr-model-tfidf.csv', index=False)

In [94]:
y_pred_real_lr_tfidf = BNBmodel.predict(X_real_test_tfidf)
data = {'id':test_data.id,'text':X_real_test_raw,'sentiment':y_pred_real_lr_tfidf}
predictions = pd.DataFrame(data=data)
(predictions.drop(['text'], axis=1)).to_csv('bnb-model-tfidf.csv', index=False)

In [95]:
# #Importing required libraries
# from sklearn.model_selection import KFold 
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import cross_val_score
# from numpy import mean
# from numpy import std

# # k = 5
# # kf = KFold(n_splits=k, random_state=None)
# # model = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1)
 
# # acc_score = []
 
# # for train_index , test_index in kf.split(X_train_tfidf):
# #     X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
# #     y_train , y_test = y[train_index] , y[test_index]
     
# #     model.fit(X_train,y_train)
# #     pred_values = model.predict(X_test)
     
# #     acc = accuracy_score(pred_values , y_test)
# #     acc_score.append(acc)
     
# # avg_acc_score = sum(acc_score)/k
 
# # print('accuracy of each fold - {}'.format(acc_score))
# # print('Avg accuracy : {}'.format(avg_acc_score))

# cv = KFold(n_splits=100, random_state=1, shuffle=True)
# # create model
# model = LogisticRegression()
# # evaluate model
# scores = cross_val_score(model, X_train_tfidf, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
# # report performance
# print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [96]:
# # compare ensemble to each baseline classifier
# from numpy import mean
# from numpy import std
# from sklearn.datasets import make_classification
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import RepeatedStratifiedKFold
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
# from sklearn.ensemble import StackingClassifier
# from matplotlib import pyplot
 
# # get a stacking ensemble of models
# def get_stacking():
# 	# define the base models
# 	level0 = list()
# 	level0.append(('lr', LogisticRegression(multi_class='multinomial')))
# 	level0.append(('bayes', MultinomialNB()))
# 	level0.append(('rf', RandomForestClassifier(n_estimators=200, random_state=0)))
# 	# define meta learner model
# 	level1 = LogisticRegression(multi_class='multinomial')
# 	# define the stacking ensemble
# 	model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
# 	return model
 
# # get a list of models to evaluate
# def get_models():
# 	models = dict()
# 	models['lr'] = LogisticRegression(multi_class='multinomial')
# 	models['bayes'] = MultinomialNB()
# 	models['rf']= RandomForestClassifier(n_estimators=200, random_state=0)
# 	models['stacking'] = get_stacking()
# 	return models
 
# # evaluate a give model using cross-validation
# def evaluate_model(model, X, y):
# 	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# 	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# 	return scores
 
# # define dataset
# X = X_train_tfidf
# y = y_train
# # get the models to evaluate
# models = get_models()
# # evaluate the models and store results
# results, names = list(), list()
# for name, model in models.items():
# 	scores = evaluate_model(model, X, y)
# 	results.append(scores)
# 	names.append(name)
# 	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# # plot model performance for comparison
# pyplot.boxplot(results, labels=names, showmeans=True)
# pyplot . show()